## Dependencies

In [1]:
import numpy as np
import pandas as pd

from utils import *

## I/O - Initialization

In [9]:
# load the feature dataset as a dataframe
csv_file = 'eeg_features.csv'
channel_list = ['F3', 'F4','C3','Cz','Oz']
selected_channels, selected_labels = data_loader(path = csv_file, channel_list = channel_list, ds = False, ds_rate = 1)

In [10]:
# select the models to be trained
#models = ['GBC', 'K-NN', 'SVM', 'DTC', 'RFC', 'Logistic Regression', 'NN']
#models = ['K-NN', 'SVM']
#data = data_preparation(selected_channels=selected_channels, selected_labels=selected_labels, feature_subset=selected_channels.columns)

## P-Value Thresholding

In [11]:
all = feature_selection(selected_channels=selected_channels, feature_subset=selected_channels.columns) # select every feature
p_all, p_dict = p_value_thresholding(selected_features=all, selected_labels=selected_labels)

## Feature Ranking

In [12]:
# training with the features that rank highest on the variance thresholded feature subset
stop_feature = 'peak_freq'
high_rank_subset = p_value_slicing(p_values=p_all, stop_feature=stop_feature)
#data = data_preparation(selected_channels=data, selected_labels=selected_labels, feature_subset=high_rank_subset)

#for model in models:
#    model_training(data, model, display_labels, stats=True, cm=False)